<a href="https://colab.research.google.com/github/saai07/LLM_finetuning/blob/main/llama-3.1-8B-ascii-cats-lora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

# load model

In [1]:
from unsloth import  FastLanguageModel
import torch
from google.colab import userdata
!pip install modelscope
import os; os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name= "meta-llama/llama-3.2-3B",
    max_seq_length= 2048,
    dtype = None,
    load_in_4bit= False,
    token = userdata.get('HF_ACCESS_TOKEN')
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Add lora adapters

In [3]:
target_modules =  ["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"]
train_embeddings = False

if train_embeddings:
  target_modules = target_modules + ['lm_head']

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=target_modules,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    random_state = 43,
    use_rslora=False,
    use_gradient_checkpointing='unsloth',
    loftq_config=None

)

Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


NameError: name 'trainer' is not defined

In [4]:
empty_promt ="""
{ascii_art}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_promts_func_no_promts(examples):
  ascii_art_sample = examples["ascii"]
  training_prompts_list = [] # Initialize as a list to store all formatted prompts
  for ascii_art in ascii_art_sample:
    formatted_prompt = empty_promt.format(ascii_art=ascii_art) + EOS_TOKEN
    training_prompts_list.append(formatted_prompt)
  return {"text": training_prompts_list} # Return the list outside the loop

from datasets import load_dataset
dataset = load_dataset("pookie3000/ascii-cats", split = "train")
dataset = dataset.map(formatting_promts_func_no_promts, batched= True)

# Visualize dataset

In [5]:
for i , sample in enumerate(dataset):
    print(f" \n -------------- Sample {i+1}-------")
    print(sample["text"])
    if i>3:
      break

 
 -------------- Sample 1-------

    /\_/\           ___
   = o_o =_______    \ \ 
    __^      __(  \.__) )
(@)<_____>__(_____)____/
<|end_of_text|>
 
 -------------- Sample 2-------

|\---/|
| o_o |
 \_^_/
<|end_of_text|>
 
 -------------- Sample 3-------

 |\__/,|   (`\
 |_ _  |.--.) )
 ( T   )     /
(((^_(((/(((_/
<|end_of_text|>
 
 -------------- Sample 4-------

   |\---/|
   | ,_, |
    \_`_/-..----.
 ___/ `   ' ,""+ \  
(__...'   __\    |`.___.';
  (_,...'(_,.`__)/'.....+
<|end_of_text|>
 
 -------------- Sample 5-------

      /\_/\
 /\  / o o \
//\\ \~(*)~/
`  \/   ^ /
   | \|| ||
   \ '|| ||
    \)()-())
<|end_of_text|>


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer =SFTTrainer(
    model =model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc =2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,
        torch_compile = False,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed =43,
        output_dir = "output",
        report_to = "none",
        push_to_hub = False
    ),

)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/201 [00:00<?, ? examples/s]

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 201 | Num Epochs = 5 | Total steps = 130
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,4.201300
2,3.704700
3,4.132600
4,3.716800
5,3.801100
6,4.061500
7,3.678500
8,3.446700
9,3.596600
10,3.575800


#infrence


In [8]:
from transformers import TextStreamer

def generate_ascii_art(model):
  FastLanguageModel.for_inference(model)
  input = tokenizer("", return_tensors="pt").to("cuda")
  text_streamer = TextStreamer(tokenizer)
  for token in model.generate(**input  ,streamer = text_streamer , max_new_tokens =100):
    print(token)
    pass


In [9]:
for _ in range(4):
  generate_ascii_art(model)

<|begin_of_text|>
  /\_/\  (
 ( ~.~ ) _)
   \"/  (
 ( | | )
(__d b__)
<|end_of_text|>
tensor([128000,    198,    220,  24445,     62,  35419,    220,   2456,    320,
          4056,     13,     93,    883,    721,    340,    256,   1144,   3193,
           220,   2456,    320,    765,    765,   1763,   5630,     67,    293,
         24010, 128001], device='cuda:0')
<|begin_of_text|>
  /\ ___ /\
 (  >   <  )
  \  >#<  /
  /       \
 /         \
 |         |
  \       /
  /// /// --
<|end_of_text|>
tensor([128000,    198,    220,  24445,   7588,    611,   5779,    320,    220,
           871,    256,    366,    220,   1763,    220,   1144,    220,    871,
             2,     27,    220,  40081,    220,    611,    996,   3120,    611,
           260,   3120,    765,    260,   9432,    220,   1144,    996,  40081,
           220,   1066,   1066,  40614, 128001], device='cuda:0')
<|begin_of_text|>
       /\_/\  
       ( >.< )  
        > ^ < 
        \ .\  
         / .\  
        (   (  


# Saving

In [15]:
from google.colab import userdata

if False:
  model.push_to_hub_gguf("sai237/llama-3.1-8B-ascii-cats-lora", tokenizer , quantization_method= "q4_0" , token= userdata.get("HF_ACCESS_TOKEN"))

model.push_to_hub("sai237/llama-3.1-8B-ascii-cats-lora" , tokenizer , token= userdata.get("HF_ACCESS_TOKEN"))

README.md:   0%|          | 0.00/528 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 45.8kB / 97.3MB            

Saved model to https://huggingface.co/sai237/llama-3.1-8B-ascii-cats-lora


In [14]:
from huggingface_hub import login
login()
